## Importing the required libraries

In [1]:
from matplotlib import pyplot as plt
from skimage.transform import AffineTransform, warp
from skimage import io, img_as_ubyte
import random
from scipy.ndimage import rotate
import albumentations as A
import glob
import os
from keras.metrics import MeanIoU
import numpy as np
import imageio
import tensorflow as tf
from tensorflow import keras
from keras.regularizers import l1
import boto3
import shutil
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras.optimizers import Adam

## Pre-processing the tiff files

In [ ]:
images_to_generate=10000
images_path="E:/Chancy/Urban_Heat/original/img/" #path to original images
masks_path = "E:/Chancy/Urban_Heat/original/mask/"
img_augmented_path="E:/Chancy/Urban_Heat/aug/img/" # path to store aumented images
msk_augmented_path="E:/Chancy/Urban_Heat/aug/mask/" # path to store aumented images
images=[] # to store paths of images from folder
masks=[]
for im in os.listdir(images_path):  # read image name from folder and append its path into "images" array
    images.append(os.path.join(images_path,im))
for msk in os.listdir(masks_path):  # read image name from folder and append its path into "images" array
    masks.append(os.path.join(masks_path,msk))
aug = A.Compose([
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.HorizontalFlip(p=1),
    A.Transpose(p=1),
    A.GridDistortion(p=1)
    ]
)

i=1   # variable to iterate till images_to_generate
while i<=images_to_generate:
    number = random.randint(0, len(images)-1)  #Pick a number to select an image & mask
    image = images[number]
    mask = masks[number]
    print(image, mask)
    image=random.choice(images) #Randomly select an image name
    original_image = io.imread(image)
    original_image = np.resize(original_image, (256, 256, 3))
    original_mask = io.imread(mask)
    original_mask = np.resize(original_mask, (256, 256, 3))
    augmented = aug(image=original_image, mask=original_mask)
    transformed_image = augmented['image']
    transformed_mask = augmented['mask']
    new_image_path= "%s/augmented_image_%s.png" %(img_augmented_path, i)
    new_mask_path = "%s/augmented_mask_%s.png" %(msk_augmented_path, i)
    io.imsave(new_image_path, transformed_image)
    io.imsave(new_mask_path, transformed_mask)
    i =i+1



## Downloading data from s3 bucket

In [ ]:
s3 = boto3.client('s3')
s3_resource = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id='',
    aws_secret_access_key=''
)

In [ ]:
def downloadDirectoryFroms3(bucketName, remoteDirectoryName):
    bucket = s3_resource.Bucket(bucketName) 
    for obj in bucket.objects.filter(Prefix = remoteDirectoryName):
        if not os.path.exists(os.path.dirname(obj.key)):
            os.makedirs(os.path.dirname(obj.key))
        try:
            bucket.download_file(obj.key, obj.key) # save to same path
        except:
            pass

In [ ]:
downloadDirectoryFroms3('aws-disaster', 'img6')
downloadDirectoryFroms3('aws-disaster', 'mask6')

## Batching data for training

In [ ]:
os.mkdir('images4')
os.mkdir('masks4')

src_mask = './mask6/'
dest_mask = './masks4/'
for files in os.listdir(src_mask):
    try:
        if int(files.split('.')[0].split('_')[2]) > 9000 and int(files.split('.')[0].split('_')[2]) < 10001:
            # print(files)
            shutil.copy(os.path.join(src_mask,files), dest_mask)
    except:
        pass


src_img = './img6/'
dest_img = './images4/'
for files in os.listdir(src_img):
    # print(files)
    try:
        if int(files.split('.')[0].split('_')[2]) > 9000 and int(files.split('.')[0].split('_')[2]) < 10001:
            # print(files.split('.')[0])
            shutil.copy(os.path.join(src_img,files), dest_img)
    except:
        pass


## Asserting same number of images and masks

In [ ]:
im = mask = 0
for img_path in glob.glob(os.path.join('./images4',"*.png")):
    im += 1

for img_path in glob.glob(os.path.join('./masks4',"*.png")):
    mask += 1

print(im,mask)

## Converting images and masks to array

In [ ]:
train_images = []
# for directory_path in glob.glob('./aug-20220204T063105Z-001/aug'):
for img_path in glob.glob(os.path.join('./images4',"*.png")):
    #print(img_path)
    img = cv2.imread(img_path, 0)
    train_images.append(img)
#Convert list to array for machine learning processing
train_images = np.array(train_images)
# print(train_images)

train_masks= []
# for directory_path in glob.glob(msk_augmented_path):
for img_path in glob.glob(os.path.join('./masks4', "*.png")):
    #print(img_path)
    mask = cv2.imread(img_path,0)
    train_masks.append(mask)
#Convert list to array for machine learning processing
train_masks = np.array(train_masks)


np.unique(train_masks)

## Pre-processing before training

In [ ]:
train_images = np.expand_dims(train_images, axis=3)
train_images = tf.keras.utils.normalize(train_images, axis=1)

train_masks_input = np.expand_dims(train_masks, axis=3)

## Training into train and validation

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(train_images, train_masks_input, test_size = 0.20, random_state = 0)

In [ ]:
n_classes=4
train_masks_cat = tf.keras.utils.to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))



test_masks_cat = tf.keras.utils.to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

## Modified U-net Model

In [ ]:
def multi_unet_model(n_classes=4, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=1):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs
    #Contraction path
    c1 = Conv2D(16, (3, 3), kernel_regularizer=l1(0.01),  kernel_initializer='he_uniform', padding='same')(s)
    c1 = tf.keras.layers.LeakyReLU(alpha=0.1)(c1)
    c1 = BatchNormalization()(c1)
    c1 = Dropout(0.25)(c1)
    c1 = Conv2D(16, (3, 3), kernel_regularizer=l1(0.01) ,kernel_initializer='he_uniform', padding='same')(c1)
    c1 = tf.keras.layers.LeakyReLU(alpha=0.1)(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), kernel_regularizer=l1(0.01), kernel_initializer='he_uniform', padding='same')(p1)
    c2 = tf.keras.layers.LeakyReLU(alpha=0.1)(c2)
    c2 = BatchNormalization()(c2)
    c2 = Dropout(0.25)(c2)
    c2 = Conv2D(32, (3, 3), kernel_regularizer=l1(0.01), kernel_initializer='he_uniform', padding='same')(c2)
    c2 = tf.keras.layers.LeakyReLU(alpha=0.1)(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), kernel_regularizer=l1(0.01) ,kernel_initializer='he_uniform', padding='same')(p2)
    c3 = tf.keras.layers.LeakyReLU(alpha=0.1)(c3)
    c3 = BatchNormalization()(c3)
    c3 = Dropout(0.25)(c3)
    c3 = Conv2D(64, (3, 3), kernel_regularizer=l1(0.01), kernel_initializer='he_uniform', padding='same')(c3)
    c3 = tf.keras.layers.LeakyReLU(alpha=0.1)(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3),kernel_regularizer=l1(0.01),  kernel_initializer='he_uniform', padding='same')(p3)
    c4 = tf.keras.layers.LeakyReLU(alpha=0.1)(c4)
    c4 = BatchNormalization()(c4)
    #c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3),kernel_regularizer=l1(0.01),  kernel_initializer='he_uniform', padding='same')(c4)
    c4 = tf.keras.layers.LeakyReLU(alpha=0.1)(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3),kernel_regularizer=l1(0.01), kernel_initializer='he_uniform', padding='same')(p4)
    c5 = tf.keras.layers.LeakyReLU(alpha=0.1)(c5)
    c5 = BatchNormalization()(c5)
    # c5 = Dropout(0.2)(c5)
    c5 = Conv2D(256, (3, 3), kernel_regularizer=l1(0.01), kernel_initializer='he_uniform', padding='same')(c5)
    c5 = tf.keras.layers.LeakyReLU(alpha=0.1)(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (3 ,3), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3),kernel_regularizer=l1(0.01),  kernel_initializer='he_uniform', padding='same')(u6)
    c6 = tf.keras.layers.LeakyReLU(alpha=0.1)(c6)
    c6 = BatchNormalization()(c6)
    #c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3),kernel_regularizer=l1(0.01),  kernel_initializer='he_uniform', padding='same')(c6)
    c6 = tf.keras.layers.LeakyReLU(alpha=0.1)(c6)
     
    u7 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), kernel_regularizer=l1(0.01), kernel_initializer='he_uniform', padding='same')(u7)
    c7 = tf.keras.layers.LeakyReLU(alpha=0.1)(c7)
    c7 = BatchNormalization()(c7)
    # c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3),kernel_regularizer=l1(0.01),  kernel_initializer='he_uniform', padding='same')(c7)
    c7 = tf.keras.layers.LeakyReLU(alpha=0.1)(c7)
     
    u8 = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3),kernel_regularizer=l1(0.01),  kernel_initializer='he_uniform', padding='same')(u8)
    c8 = tf.keras.layers.LeakyReLU(alpha=0.1)(c8)
    c8 = BatchNormalization()(c8)
    # c8 = Dropout(0.2)(c8)
    c8 = Conv2D(32, (3, 3),kernel_regularizer=l1(0.01),  kernel_initializer='he_uniform', padding='same')(c8)
    c8 = tf.keras.layers.LeakyReLU(alpha=0.1)(c8)
     
    u9 = Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), kernel_regularizer=l1(0.01), kernel_initializer='he_uniform', padding='same')(u9)
    c9 = tf.keras.layers.LeakyReLU(alpha=0.1)(c9)
    c9 = BatchNormalization()(c9)
    # c9 = Dropout(0.2)(c9)
    c9 = Conv2D(16, (3, 3), kernel_regularizer=l1(0.01),  kernel_initializer='he_uniform', padding='same')(c9)
    c9 = tf.keras.layers.LeakyReLU(alpha=0.1)(c9)
     
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)
     
    model = Model(inputs=[inputs], outputs=[outputs])

    
    return model

## Compiling Model

In [ ]:
IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH  = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]

def get_model():
    return multi_unet_model(n_classes=n_classes, IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS)
optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model_channel_1 = get_model()
model_channel_1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

## Training

In [ ]:
history_channel_1 = model_channel_1.fit(X_train, y_train_cat, 
                    batch_size = 16, 
                    verbose=1, 
                    epochs=50, 
                    validation_data=(X_test, y_test_cat), 
                    # class_weight=class_weights,
                    shuffle=False)

## Saving model

In [ ]:
model_channel_1.save('channel1_150_epochs.h5')

## Loading model

In [ ]:
loaded_model = tf.keras.models.load_model("channel1_150_epochs.h5")
_, acc =  loaded_model.evaluate(X_test, y_test_cat)
print("Accuracy is = ", (acc * 100.0), "%")

## Evaluation

In [ ]:
y_pred= loaded_model.predict(X_test)
y_pred_argmax=np.argmax(y_pred, axis=3)


y_pred_argmax


In [ ]:
n_classes = 4
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test[:,:,:,0], y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())



values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
print(values)
class1_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[0,3] + values[1,0]+ values[2,0]+ values[3,0])
class2_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[1,3] + values[0,1]+ values[2,1]+ values[3,1])
class3_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[2,3] + values[0,2]+ values[1,2]+ values[3,2])
class4_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])

print("IoU for class1 is: ", class1_IoU)
print("IoU for class2 is: ", class2_IoU)
print("IoU for class3 is: ", class3_IoU)
print("IoU for class4 is: ", class4_IoU)

plt.imshow(train_images[0, :,:,0], cmap='gray')
plt.imshow(train_masks[0], cmap='gray')


## Plotting the results

In [ ]:

test_img_number = random.randint(0, len(X_test))
test_img = X_test[test_img_number]
ground_truth=y_test[test_img_number]
test_img_norm=test_img[:,:,0][:,:,None]
test_img_input=np.expand_dims(test_img_norm, 0)
prediction = (loaded_model.predict(test_img_input))
predicted_img=np.argmax(prediction, axis=3)[0,:,:]


plt.figure(figsize=(12, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth[:,:,0], cmap='gray')
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(predicted_img, cmap='gray')
plt.show()

## Predictions

In [ ]:


def prediction(image, patch_size):
    segm_img = np.zeros(image.shape[:2])  #Array with zeros to be filled with segmented values
    patch_num=1
    for i in range(0, image.shape[0]):   #Steps of 256
        for j in range(0, image.shape[1]):  #Steps of 256
            #print(i, j)
            single_patch = image[i:i+patch_size, j:j+patch_size]
            single_patch_norm = np.expand_dims(tf.keras.utils.normalize(np.array(single_patch), axis=1),2)
            single_patch_shape = single_patch_norm.shape[:2]
            single_patch_input = np.expand_dims(single_patch_norm, 0)
            single_patch_prediction = (model.predict(single_patch_input)[0,:,:,0] > 0.5).astype(np.uint8)
            segm_img[i:i+single_patch_shape[0], j:j+single_patch_shape[1]] += cv2.resize(single_patch_prediction, single_patch_shape[::-1])
          
            print("Finished processing patch number ", patch_num, " at position ", i,j)
            patch_num+=1
    return segm_img

In [ ]:
large_image = cv2.imread('satellite-venice-italy_1.jpg', 0)

print(large_image.shape)

prediction(large_image, 256)